<a href="https://colab.research.google.com/github/taciturn02/CreditCardFruadDetection/blob/main/ML_MODELS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [236]:
from google.colab import drive

In [237]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [238]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score,classification_report,confusion_matrix,f1_score,roc_auc_score
from sklearn.model_selection import KFold,cross_val_score



In [239]:
train_df = pd.read_csv("/content/drive/MyDrive/CreditCard_DataSet/randomUndersammpledData.csv")
train_df2 = pd.read_csv("/content/drive/MyDrive/CreditCard_DataSet/randomOversampledData.csv")

In [240]:
train_df.info()
train_df2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4290 entries, 0 to 4289
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   merchant    4290 non-null   int64  
 1   category    4290 non-null   int64  
 2   amt         4290 non-null   float64
 3   gender      4290 non-null   int64  
 4   lat         4290 non-null   float64
 5   long        4290 non-null   float64
 6   city_pop    4290 non-null   int64  
 7   merch_lat   4290 non-null   float64
 8   merch_long  4290 non-null   float64
 9   is_fraud    4290 non-null   int64  
 10  hour        4290 non-null   int64  
 11  day         4290 non-null   int64  
 12  month       4290 non-null   int64  
 13  age_cust    4290 non-null   int64  
dtypes: float64(5), int64(9)
memory usage: 469.3 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1107148 entries, 0 to 1107147
Data columns (total 14 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      ------------

Split data into target and features

In [241]:
from sklearn.preprocessing import MinMaxScaler


In [242]:
XX = train_df.drop(columns='is_fraud', axis=1)
YY = train_df['is_fraud']

In [243]:
scaler = MinMaxScaler()
X= scaler.fit_transform(XX)


In [244]:
X.mean(axis=0)

array([0.48958864, 0.52404519, 0.11581572, 0.45734266, 0.40928096,
       0.77038171, 0.02866145, 0.4132861 , 0.764363  , 0.5854059 ,
       0.49273504, 0.5506216 , 0.39971222])

In [245]:
X

array([[0.6632948 , 0.30769231, 0.02165597, ..., 0.66666667, 1.        ,
        0.37037037],
       [0.34104046, 0.        , 0.0198289 , ..., 1.        , 0.        ,
        0.43209877],
       [0.03323699, 0.76923077, 0.01309331, ..., 0.66666667, 0.33333333,
        0.54320988],
       ...,
       [0.64017341, 0.84615385, 0.40452909, ..., 0.83333333, 1.        ,
        0.56790123],
       [0.04190751, 0.30769231, 0.11229626, ..., 0.83333333, 1.        ,
        0.56790123],
       [0.40606936, 0.61538462, 0.29816203, ..., 0.83333333, 1.        ,
        0.56790123]])

Initalize K-Fold Cross Validation

In [246]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)


In [247]:
metrics = {
    'accuracy': accuracy_score,
    'precision': precision_score,
    'f1_macro' : f1_score,
    'recall': recall_score,
    'roc_auc': roc_auc_score,

}

In [248]:
def model_testing1(estimator):
    ## Prediction on data
    y_preds = estimator.predict(X_test)

    ##Print accuracy score
    print("Accuracy Score on UnderSampled Test Data: ",accuracy_score(Y_test,y_preds))

    ## Print classification report
    print("\nConfussion Matrix  of UnderSampled Test Data:\n " ,confusion_matrix(Y_test ,y_preds),"\n")

    ## Check confusion matrix
    print("classification_report of UnderSampled Test Data: \n\n",classification_report(Y_test ,y_preds))


In [249]:
def model_testing11(estimator):
    ## Prediction on data
    y_preds = estimator.predict(X_train)

    ##Print accuracy score
    print("Accuracy Score on UnderSampled Train Data: ",accuracy_score(Y_train,y_preds))

    ## Print classification report
    print("\nConfussion Matrix  of UnderSampled Train Data:\n " ,confusion_matrix(Y_train ,y_preds),"\n")

    ## Check confusion matrix
    print("classification_report of UnderSampled Train Data: \n\n",classification_report(Y_train ,y_preds))

##LOGISTIC REGRESSION

In [250]:
model = LogisticRegression()


In [251]:
evaluation_results = {}
for metric_name, metric_func in metrics.items():
    scores = cross_val_score(model, X, Y, cv=kfold, scoring=metric_name)
    evaluation_results[metric_name] = scores.mean()

MODEL EVALUATION

In [252]:
for metric_name, score in evaluation_results.items():
    print(f"Mean_{metric_name.capitalize()}: {score}\n")

Mean_Accuracy: 0.8435897435897436

Mean_Precision: 0.9285865070465888

Mean_F1_macro: 0.8419755316601123

Mean_Recall: 0.7449889100732412

Mean_Roc_auc: 0.8616284067360113



Printing Metrics for the best Fold




In [253]:
accuracy_scores = cross_val_score(model, X, Y, cv=kfold, scoring='accuracy')
best_fold_index = np.argmax(accuracy_scores)


 Get the train and test indices for the best fold

In [254]:
train_index, test_index = list(kfold.split(X))[best_fold_index]


 Split the data into train and test sets for the best fold


In [255]:
X_train, X_test = X[train_index], X[test_index]
Y_train, Y_test = Y[train_index], Y[test_index]

In [256]:
model.fit(X_train, Y_train)

LogisticRegression()

In [257]:
model_testing1(model)


Accuracy Score on UnderSampled Test Data:  0.8601398601398601

Confussion Matrix  of UnderSampled Test Data:
  [[407  17]
 [103 331]] 

classification_report of UnderSampled Test Data: 

               precision    recall  f1-score   support

           0       0.80      0.96      0.87       424
           1       0.95      0.76      0.85       434

    accuracy                           0.86       858
   macro avg       0.87      0.86      0.86       858
weighted avg       0.88      0.86      0.86       858



In [258]:
model_testing11(model)

Accuracy Score on UnderSampled Train Data:  0.8406177156177156

Confussion Matrix  of UnderSampled Train Data:
  [[1618  103]
 [ 444 1267]] 

classification_report of UnderSampled Train Data: 

               precision    recall  f1-score   support

           0       0.78      0.94      0.86      1721
           1       0.92      0.74      0.82      1711

    accuracy                           0.84      3432
   macro avg       0.85      0.84      0.84      3432
weighted avg       0.85      0.84      0.84      3432



##DECISION TREE MODEL

In [259]:
from sklearn.tree import DecisionTreeClassifier


In [260]:

model = DecisionTreeClassifier(criterion = 'entropy',max_depth = 6, random_state=42)

In [261]:
evaluation_results = {}
for metric_name, metric_func in metrics.items():
    scores = cross_val_score(model, X, Y, cv=kfold, scoring=metric_name)
    evaluation_results[metric_name] = scores.mean()

Model Evaluation

In [262]:
for metric_name, score in evaluation_results.items():
    print(f"Mean_{metric_name.capitalize()}: {score}\n")

Mean_Accuracy: 0.941025641025641

Mean_Precision: 0.9447243749927159

Mean_F1_macro: 0.9409942587063445

Mean_Recall: 0.9373157587485277

Mean_Roc_auc: 0.9792043085926402



Printing metrices for the best fold

In [263]:
accuracy_scores = cross_val_score(model, X, Y, cv=kfold, scoring='accuracy')
best_fold_index = np.argmax(accuracy_scores)

Get the train and test indices for the best fold

In [264]:
train_index, test_index = list(kfold.split(X))[best_fold_index]

Split the data into train and test sets for the best fold

In [265]:
X_train, X_test = X[train_index], X[test_index]
Y_train, Y_test = Y[train_index], Y[test_index]

In [266]:
model.fit(X_train, Y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=42)

In [267]:
model_testing1(model)

Accuracy Score on UnderSampled Test Data:  0.9522144522144522

Confussion Matrix  of UnderSampled Test Data:
  [[431  21]
 [ 20 386]] 

classification_report of UnderSampled Test Data: 

               precision    recall  f1-score   support

           0       0.96      0.95      0.95       452
           1       0.95      0.95      0.95       406

    accuracy                           0.95       858
   macro avg       0.95      0.95      0.95       858
weighted avg       0.95      0.95      0.95       858



In [268]:
model_testing11(model)

Accuracy Score on UnderSampled Train Data:  0.9493006993006993

Confussion Matrix  of UnderSampled Train Data:
  [[1616   77]
 [  97 1642]] 

classification_report of UnderSampled Train Data: 

               precision    recall  f1-score   support

           0       0.94      0.95      0.95      1693
           1       0.96      0.94      0.95      1739

    accuracy                           0.95      3432
   macro avg       0.95      0.95      0.95      3432
weighted avg       0.95      0.95      0.95      3432



##RANDOM FOREST MODEL

In [269]:
from sklearn.ensemble import RandomForestClassifier

In [270]:
model = RandomForestClassifier(criterion = 'entropy',max_depth=10, random_state=42)


In [271]:
evaluation_results = {}
for metric_name, metric_func in metrics.items():
    scores = cross_val_score(model, X, Y, cv=kfold, scoring=metric_name)
    evaluation_results[metric_name] = scores.mean()

In [272]:
for metric_name, score in evaluation_results.items():
    print(f"Mean_{metric_name.capitalize()}: {score}\n")

Mean_Accuracy: 0.95011655011655

Mean_Precision: 0.9595455710948476

Mean_F1_macro: 0.9500749705083464

Mean_Recall: 0.9398919426004128

Mean_Roc_auc: 0.9890707450389282



Printing metrices for the best fold

In [273]:
accuracy_scores = cross_val_score(model, X, Y, cv=kfold, scoring='accuracy')
best_fold_index = np.argmax(accuracy_scores)

Get the train and test indices for the best fold

In [274]:
train_index, test_index = list(kfold.split(X))[best_fold_index]

Split the data into train and test sets for the best fold

In [275]:
X_train, X_test = X[train_index], X[test_index]
Y_train, Y_test = Y[train_index], Y[test_index]

In [276]:
model.fit(X_train, Y_train)

RandomForestClassifier(criterion='entropy', max_depth=10, random_state=42)

In [277]:
model_testing1(model)

Accuracy Score on UnderSampled Test Data:  0.9545454545454546

Confussion Matrix  of UnderSampled Test Data:
  [[416  14]
 [ 25 403]] 

classification_report of UnderSampled Test Data: 

               precision    recall  f1-score   support

           0       0.94      0.97      0.96       430
           1       0.97      0.94      0.95       428

    accuracy                           0.95       858
   macro avg       0.95      0.95      0.95       858
weighted avg       0.95      0.95      0.95       858



In [278]:
model_testing11(model)

Accuracy Score on UnderSampled Train Data:  0.9857226107226107

Confussion Matrix  of UnderSampled Train Data:
  [[1712    3]
 [  46 1671]] 

classification_report of UnderSampled Train Data: 

               precision    recall  f1-score   support

           0       0.97      1.00      0.99      1715
           1       1.00      0.97      0.99      1717

    accuracy                           0.99      3432
   macro avg       0.99      0.99      0.99      3432
weighted avg       0.99      0.99      0.99      3432



##XGBOOST CLASSIFIER MODEL

Initializie XGBoost Classifier



In [279]:
import xgboost as xgb


In [280]:
model = xgb.XGBClassifier()


In [281]:
evaluation_results = {}
for metric_name, metric_func in metrics.items():
    scores = cross_val_score(model, X, Y, cv=kfold, scoring=metric_name)
    evaluation_results[metric_name] = scores.mean()

In [282]:
for metric_name, score in evaluation_results.items():
    print(f"Mean_{metric_name.capitalize()}: {score}\n")

Mean_Accuracy: 0.9706293706293707

Mean_Precision: 0.9699351929242803

Mean_F1_macro: 0.970603516506063

Mean_Recall: 0.9715443636505278

Mean_Roc_auc: 0.9953969676954696



In [283]:
accuracy_scores = cross_val_score(model, X, Y, cv=kfold, scoring='accuracy')
best_fold_index = np.argmax(accuracy_scores)

In [284]:
train_index, test_index = list(kfold.split(X))[best_fold_index]

In [285]:
X_train, X_test = X[train_index], X[test_index]
Y_train, Y_test = Y[train_index], Y[test_index]

In [286]:
model.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [287]:
model_testing1(model)

Accuracy Score on UnderSampled Test Data:  0.9766899766899767

Confussion Matrix  of UnderSampled Test Data:
  [[444   8]
 [ 12 394]] 

classification_report of UnderSampled Test Data: 

               precision    recall  f1-score   support

           0       0.97      0.98      0.98       452
           1       0.98      0.97      0.98       406

    accuracy                           0.98       858
   macro avg       0.98      0.98      0.98       858
weighted avg       0.98      0.98      0.98       858



In [288]:
model_testing11(model)

Accuracy Score on UnderSampled Train Data:  1.0

Confussion Matrix  of UnderSampled Train Data:
  [[1693    0]
 [   0 1739]] 

classification_report of UnderSampled Train Data: 

               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1693
           1       1.00      1.00      1.00      1739

    accuracy                           1.00      3432
   macro avg       1.00      1.00      1.00      3432
weighted avg       1.00      1.00      1.00      3432



##LIGHT GBM CLASSIFIER MODEL

In [289]:
import lightgbm as lgb

In [290]:
model = lgb.LGBMClassifier()

In [291]:
evaluation_results = {}
for metric_name, metric_func in metrics.items():
    scores = cross_val_score(model, X, Y, cv=kfold, scoring=metric_name)
    evaluation_results[metric_name] = scores.mean()

[LightGBM] [Info] Number of positive: 1739, number of negative: 1693
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000267 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1892
[LightGBM] [Info] Number of data points in the train set: 3432, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.506702 -> initscore=0.026808
[LightGBM] [Info] Start training from score 0.026808
[LightGBM] [Info] Number of positive: 1711, number of negative: 1721
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001130 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1898
[LightGBM] [Info] Number of data points in the train set: 3432, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498543 -> initscore=-0.005828
[LightGBM]

In [292]:
for metric_name, score in evaluation_results.items():
    print(f"Mean_{metric_name.capitalize()}: {score}\n")

Mean_Accuracy: 0.9699300699300698

Mean_Precision: 0.972439187103039

Mean_F1_macro: 0.9699054687782145

Mean_Recall: 0.9673640602006246

Mean_Roc_auc: 0.995518301517421



In [293]:
accuracy_scores = cross_val_score(model, X, Y, cv=kfold, scoring='accuracy')
best_fold_index = np.argmax(accuracy_scores)

[LightGBM] [Info] Number of positive: 1739, number of negative: 1693
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000238 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1892
[LightGBM] [Info] Number of data points in the train set: 3432, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.506702 -> initscore=0.026808
[LightGBM] [Info] Start training from score 0.026808
[LightGBM] [Info] Number of positive: 1711, number of negative: 1721
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000752 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1898
[LightGBM] [Info] Number of data points in the train set: 3432, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498543 -> initscore=-0.005828
[LightGBM]

In [294]:
train_index, test_index = list(kfold.split(X))[best_fold_index]

In [295]:
X_train, X_test = X[train_index], X[test_index]
Y_train, Y_test = Y[train_index], Y[test_index]

In [296]:
model.fit(X_train, Y_train)

[LightGBM] [Info] Number of positive: 1739, number of negative: 1693
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000202 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1892
[LightGBM] [Info] Number of data points in the train set: 3432, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.506702 -> initscore=0.026808
[LightGBM] [Info] Start training from score 0.026808


LGBMClassifier()

In [297]:
model_testing1(model)

Accuracy Score on UnderSampled Test Data:  0.9731934731934732

Confussion Matrix  of UnderSampled Test Data:
  [[442  10]
 [ 13 393]] 

classification_report of UnderSampled Test Data: 

               precision    recall  f1-score   support

           0       0.97      0.98      0.97       452
           1       0.98      0.97      0.97       406

    accuracy                           0.97       858
   macro avg       0.97      0.97      0.97       858
weighted avg       0.97      0.97      0.97       858



In [298]:
model_testing11(model)

Accuracy Score on UnderSampled Train Data:  1.0

Confussion Matrix  of UnderSampled Train Data:
  [[1693    0]
 [   0 1739]] 

classification_report of UnderSampled Train Data: 

               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1693
           1       1.00      1.00      1.00      1739

    accuracy                           1.00      3432
   macro avg       1.00      1.00      1.00      3432
weighted avg       1.00      1.00      1.00      3432

